# Assignment 3 - Segmentation

You are expected to complete this notebook with lines of code, plots and texts. You will need to create new cells with original code or text for your analyses. This assignment has a total of 100 points and 5 questions.

Using Canvas, you will deliver the notebook file (.ipynb) with cells executed and outputs visible.
- You should use PyTorch 1.0 or later as your deep learning framework. If you need to import a different package than the ones already imported, ask the TA if you can do it.
- No other data than the dataset variables provided should be used, and training, validation and testing splits should be the same as the ones provided.
- The cell outputs present in your delivered notebook should be reproducible by us by running your notebook cells in order.
- All code must be your own work. Code cannot be copied from another source or student. You may copy code from cells that were pre-defined in this notebook if you think it is useful for use in another part of the notebook.
- All images must be generated from data generated in your code. Do NOT import/display images generated outside your code.
- Your analysis must be your own, but if you quote text or equations from another source make sure to cite the reference.
- It is assumed that PyTorch is already installed according to the CADE or the COLAB tutorial.
- For all analyses you write, in the case that you ran code to get to its conclusions, you should provide code that provides evidence for these conclusions. This code should print the numbers that are cited, or plot a graph, or print a table, or, even better, a combination of these. Do not erase or comment code and its outputs in the case you cite them in your analyses.
- You should provide evidence that your code satisfies all the constraints of the questions. For showing that numeric constraints are satisfied, provide some kind of printed number, and not only a graph.

Other notes:
- Cells should be run in order, using Shift+Enter.
- Read all the provided code cells and its comments, as it contains variables and information that you may need to use to complete the notebook.
- To create a text cell, create it with the "+" button and change its type from "Code" to "Markdown" using the top menu. To modify a text cell, double click on it.
- If you are interested, you can check detail on formatting markdown text here: https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed
- The accuracies provided as required for each question are there to make sure you work enough on each model to get a good result. Part of the grade is based on this.
- We assume a GPU of at least 4GB of memory is available. If you want to try running the assignment with a GPU that has less than that, you can try changing the argument passed when calling the ```define_gpu_to_use``` function.  If you are getting out-of-memory errors for the GPU, you may want to check what is occupying the GPU memory by using the command ```!nvidia-smi```, which gives a report of the use of the GPU. 
- A few PyTorch details to remember:
    - Remember to toggle train/eval mode for your model
    - Remember to reset the gradients with ```zero_grad()``` before each call to ```backward()```
    - Remember to check if the loss you are using receives logits or probabilities, and adapt your model output accordingly.
    - Remember to reinstantiate your model every time you are starting a new training, so that weights are reset.
    - Remember to pass to the optimizer the set of parameters for the model you want to train.

#### Initial setups

In [1]:
!pip3 install imageio
!pip3 install matplotlib
!pip3 install scikit-image
!pip3 install scikit-learn

You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#importing the libraries used in the rest of the code
import os
import gzip
import shutil
import tarfile
import imageio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import zipfile
import collections
from skimage import morphology
from skimage.measure import block_reduce
import scipy
from torch.utils.data import Dataset
import PIL
from PIL import Image
from sklearn.metrics import f1_score
import copy
import random
import math
import torch

In [3]:
#checking what kind of system you are using
try:
  import google.colab
  from google.colab import drive
  from google.colab import files
  IN_COLAB = True
except:
  IN_COLAB = False
try:
    hostname = !hostname
    if 'lab' in hostname[0] and '.eng.utah.edu' in hostname[0]:
        IN_CADE = True
    else:
        IN_CADE = False
except:
    IN_CADE = False

assert(not IN_CADE or not IN_COLAB)

In [4]:
# with this function you set the value of the environment variable CUDA_VISIBLE_DEVICES
# to set which GPU to use
# it also reserves this amount of memory for your exclusive use. This might be important for 
# not having other people using the resources you need in shared systems
# the homework was tested in a GPU with 4GB of memory, and running this function will require at least
# as much
# if you want to test in a GPU with less memory, you can call this function
# with the argument minimum_memory_mb specifying how much memory from the GPU you want to reserve
def define_gpu_to_use(minimum_memory_mb = 3800):
    gpu_to_use = None
    try: 
        os.environ['CUDA_VISIBLE_DEVICES']
        print('GPU already assigned before: ' + str(os.environ['CUDA_VISIBLE_DEVICES']))
        return
    except:
        pass
    torch.cuda.empty_cache()
    for i in range(16):
        free_memory = !nvidia-smi --query-gpu=memory.free -i $i --format=csv,nounits,noheader
        if free_memory[0] == 'No devices were found':
            break
        free_memory = int(free_memory[0])
        if free_memory>minimum_memory_mb-500:
            gpu_to_use = i
            break
    if gpu_to_use is None:
        print('Could not find any GPU available with the required free memory of ' +str(minimum_memory_mb) + 'MB. Please use a different system for this assignment.')
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_to_use)
        print('Chosen GPU: ' + str(gpu_to_use))
        x = torch.rand((256,1024,minimum_memory_mb-500)).cuda()
        x = torch.rand((1,1)).cuda()
        del x

In [5]:
if IN_CADE:
    #setting the gpu that will be used, testing if it has enough available memory, and reserving the needed memory
    define_gpu_to_use()

Could not find any GPU available with the required free memory of 3800MB. Please use a different system for this assignment.


#### Defining functions used to load the dataset

We are going to use datasets of retinal digital images and segment the blood vessels from it. We are going to use the DRIVE dataset (https://www.isi.uu.nl/Research/Databases/DRIVE/) and the STARE dataset (http://cecas.clemson.edu/~ahoover/stare/) together in this assignment. We are going to use our own train/validation/test splits, even though the DRIVE dataset provides its own split. Please download the files named "DRIVE.zip" (register in https://www.isi.uu.nl/Research/Databases/DRIVE/download.php and follow the instructions), "stare-images.tar" (http://cecas.clemson.edu/~ahoover/stare/probing/stare-images.tar), "labels-vk.tar" (http://cecas.clemson.edu/~ahoover/stare/probing/labels-vk.tar) and put in the same folder as this notebook file so that the dataset can be loaded. 

The dataset also contains masks since the images are not shaped as squares originally, but are only padded so that we can fit them to a traditional CNN. The masks contain the information of where the original image is and where the padding is located. These masks should be used to limit where outputs are backpropagated for training and what region of the image should be used for scoring. In the mask, 1 means that it is part of the original image, and 0 means that it in the zero-padded region.

In [6]:
#delete small regions (<size) of binary images
def remove_small_regions(img, size):
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

In [7]:
#resize the images of the dataset to be half the height and half the width of the original images, so 
# that models states can fit on the GPU memory
def resize_img(img):
    if len(img.shape)==3:
        img = np.array(Image.fromarray(img).resize(((img.shape[1]+1)//2,(img.shape[0]+1)//2), PIL.Image.BILINEAR))
    else:
        img = block_reduce(img, block_size=(2, 2), func=np.max)
    return img

In [8]:
#unzips, loads and calculates masks for images from the stare dataset
def stare_read_images(tar_filename, dest_folder, do_mask = False):
    tar = tarfile.open(tar_filename)
    tar.extractall(dest_folder)
    tar.close()
    all_images = []
    all_masks = []
    for item in sorted(os.listdir(dest_folder)):
        if item.endswith('gz'):
            with gzip.open(dest_folder + item, 'rb') as f_in:
                with open(dest_folder + item[:-3], 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            os.remove(dest_folder + item) 
            img = imageio.imread(dest_folder + item[:-3])
            if len(img.shape) == 3:
                img = np.pad(img , ((1,2), (2,2),(0,0)), mode = 'constant')
            else:
                img = np.pad(img , ((1,2), (2,2)), mode = 'constant')
            img = resize_img(img)
            img = img/255.
            img = img.astype(np.float32)
            if len(img.shape) == 2:
                img = img.astype(np.float32)
                img = np.expand_dims(img, axis = 2)
            all_images.append(img)
            if do_mask:
                mask = (1-remove_small_regions(np.prod((img<50/255.)*1.0, axis = 2)>0.5, 1000))*1.0
                mask = np.expand_dims(mask, axis = 2)
                all_masks.append(mask.astype(np.float32))
    if do_mask:
        return all_images, all_masks
    else:
        return all_images

In [9]:
#unzips and loads masks for images from the stare dataset
def drive_read_images(filetype, dest_folder):
    zip_ref = zipfile.ZipFile('DRIVE.zip', 'r')
    zip_ref.extractall('datasets/drive')
    zip_ref.close()
    all_images = []
    for item in sorted(os.listdir(dest_folder)):
        if item.endswith(filetype):
            img = imageio.imread(dest_folder + item)
            if len(img.shape) == 3:
                img = np.pad(img , ((12,12), (69,70),(0,0)), mode = 'constant')
            else:
                img = np.pad(img , ((12,12), (69,70)), mode = 'constant')
            img = resize_img(img)
            img = img/255.
            img = img.astype(np.float32)
            if len(img.shape) == 2:
                img = img.astype(np.float32)
                img = np.expand_dims(img, axis = 2)
            all_images.append(img)
    return all_images

In [10]:
#load all images and put them on a list of list of arrays.
# on the inner lists, first element is an input image, second element is a segmentation groundtruth
# and third element is a mask to show where the input image is valid, in contrast to where it was padded
def get_retina_array():
    stare_images, stare_mask = stare_read_images("stare-images.tar", 'datasets/stare/images/', do_mask = True)            
    stare_segmentation = stare_read_images("labels-vk.tar", 'datasets/stare/segmentations/')   
    drive_training_images = drive_read_images('tif', 'datasets/drive/DRIVE/training/images/')
    drive_test_images = drive_read_images('tif', 'datasets/drive/DRIVE/test/images/')
    drive_training_segmentation = drive_read_images('gif', 'datasets/drive/DRIVE/training/1st_manual/')
    drive_test_segmentation = drive_read_images('gif', 'datasets/drive/DRIVE/test/1st_manual/')
    drive_training_mask = drive_read_images('gif', 'datasets/drive/DRIVE/training/mask/')
    drive_test_mask = drive_read_images('gif', 'datasets/drive/DRIVE/test/mask/')
    return list(zdip(stare_images+drive_training_images+drive_test_images, 
                           stare_segmentation+drive_training_segmentation+drive_test_segmentation, 
                           stare_mask + drive_training_mask + drive_test_mask))

In [11]:
#split can be 'train', 'val', and 'test'
#this is the function that splits a dataset into training, validation and testing set
#We are using a split of 70%-10%-20%, for train-val-test, respectively
#this function is used internally to the defined dataset classes
def get_split(array_to_split, split):
    np.random.seed(0)
    np.random.shuffle(array_to_split)
    np.random.seed()
    if split == 'train':
        array_to_split = array_to_split[:int(len(array_to_split)*0.7)]
    elif split == 'val':
        array_to_split = array_to_split[int(len(array_to_split)*0.7):int(len(array_to_split)*0.8)]
    elif split == 'test':
        array_to_split = array_to_split[int(len(array_to_split)*0.8):]
    return array_to_split

In [12]:
#for segmentations tasks, the exact transformations that are applied to 
# the input image should be applied, down to the random number used, should
# also be applied to the ground truth and to the masks. We redefine a few of
# PyTorch classes 

#apply transoforms to all tensors in list x 
def _iterate_transforms(transform, x):
    for i, xi in enumerate(x):
        x[i] = transform(x[i])
    return x

#redefining composed transform so that it uses the _iterate_transforms function
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, x):
        for transform in self.transforms:
            x = _iterate_transforms(transform, x) 
        return x

#class to rerandomize the vertical flip transformation   
class RandomVerticalFlipGenerator(object):
    def __call__(self, img):
        self.random_n = random.uniform(0, 1)
        return img

#class to perform vertical flip using randomization provided by gen
class RandomVerticalFlip(object):
    def __init__(self, gen):
        self.gen = gen

    def __call__(self, img):
        if self.gen.random_n < 0.5:
            return torch.flip(img, [2])
        return img

#class to rerandomize the horizontal flip transformation   
class RandomHorizontalFlipGenerator(object):
    def __call__(self, img):
        self.random_n = random.uniform(0, 1)
        return img

#class to perform horizontal flip using randomization provided by gen
class RandomHorizontalFlip(object):
    def __init__(self, gen):
        self.gen = gen

    def __call__(self, img):
        if self.gen.random_n < 0.5:
            return torch.flip(img, [1])
        return img 

In [13]:
#Dataset class for the retina dataset
# each item of the dataset is a tuple with three items:
# - the first element is the input image to be segmented 
# - the second element is the segmentation ground truth image 
# - the third element is a mask to know what parts of the input image should be used (for training and for scoring)
class RetinaDataset(Dataset):
    def transpose_first_index(self, x):
        x2 = (np.transpose(x[0], [2,0,1]), np.transpose(x[1], [2,0,1]), np.transpose(x[2], [2,0,1]))
        return x2
    
    def __init__(self, retina_array, split = 'train', do_transform=False):
        indexes_this_split = get_split(np.arange(len(retina_array), dtype = np.int), split)
        self.retina_array = [self.transpose_first_index(retina_array[i]) for i in indexes_this_split]
        self.split = split
        self.do_transform = do_transform
    def __getitem__(self, index):
        sample = [torch.FloatTensor(x) for x in self.retina_array[index]]
        if self.do_transform:
            v_gen = RandomVerticalFlipGenerator()
            h_gen = RandomHorizontalFlipGenerator()
            t = Compose([
                v_gen,
                RandomVerticalFlip(gen=v_gen),
                h_gen,
                RandomHorizontalFlip(gen=h_gen),
            ])
            sample = t(sample)
        return sample
    
    def __len__(self):
        return len(self.retina_array)

#### Loading and visualizing the dataset

In [14]:
retina_array = get_retina_array()
#datase to use for training:
train_dataset = RetinaDataset(retina_array, do_transform = True)
#dataset to use for validation
val_dataset = RetinaDataset(retina_array, split = 'val')
#dataset to use for testing
test_dataset = RetinaDataset(retina_array, split = 'test')

/home/u0579755/env_dir/lib/python3.6/site-packages/imageio/plugins/_tifffile.py:8388: UserWarning: unexpected end of LZW stream (code 0)
  warnings.warn('unexpected end of LZW stream (code %i)' % code)


NameError: name 'zdip' is not defined

In [ ]:
#Visualing a few cases in the training set
for batch_idx, (data, segmentation, mask) in enumerate( RetinaDataset(retina_array)):
    if batch_idx%15 == 0: 
        plt.figure()
        plt.title("Input Image")
        plt.imshow(data[:,:,:].permute([1,2,0]).cpu().numpy())
        plt.figure()
        plt.title("Segmentation ground truth")
        plt.imshow(segmentation[0,:,:].cpu().numpy())
        plt.figure()
        plt.title("Mask")
        plt.imshow(mask[0,:,:].cpu().numpy())
    

#### Defining a scoring function

To measure how good our segmentation results are, we are going to use the F1 score. This score is the harmonic mean between precision and recall, considering the foreground as the positive class and the background as the negative class. The score goes from 0 to 1, with 1 the best score possible. 

In [ ]:
#use this function to score your models
def get_score_model(model, data_loader):
    #toggle model to eval mode
    model.eval()
    
    #turn off gradients since they will not be used here
    # this is to make the inference faster
    with torch.no_grad():
        logits_predicted = np.zeros([0, 1, 304, 352])
        segmentations = np.zeros([0, 1, 304, 352])
        #run through several batches, does inference for each and store inference results
        # and store both target labels and inferenced scores 
        for image, segmentation, mask  in data_loader:
            image = image.cuda()
            logit_predicted = model(image)
            logits_predicted = np.concatenate((logits_predicted, logit_predicted.cpu().detach().numpy()*mask.numpy()), axis = 0)
            segmentations = np.concatenate((segmentations, segmentation.cpu().detach().numpy()*mask.numpy()), axis = 0)   
    #returns a list of scores, one for each of the labels
    return f1_score(segmentations.reshape([-1]), logits_predicted.reshape([-1])>0)

#### Q1 (13 points)
Justify why data augmentation usually helps in improving scores in deep learning tasks, and why horizontal flipping and vertical flipping make sense for this dataset. Would they both make sense in a natural image dataset? List at least one other kind of data augmentation that could also be applied for the retina blood vessels segmentation, and justify why.

### Why data augmentation improves scores
Data augmentation helps improve scores because it "can teach the network the desiere invariance and robustenss properties, when only few training samples are avialable" [1]  

Horizontally and vertically flipping make sense for this dataset because those invariants already exist. The optic nerve is a good landmark to recognize that the dataset includes these different invariants already; it can be seen on the right or left in different images as well as closer to the top or bottom in others.  

Vertical flipping would not make sense for natural images. Think of an image of a cat sitting on a ground. If vertically flipped, the cat would now be upside down and the sky and ground would be flipped. The upside down cat is unlikely (never going) to occur in the test dataset or in future predictions. The upside down cat is not an invariant we would want our CNN to learn.

Rotatoing the images left or right slightly. This is a similar invariant to the flipping where there is no expected location for the optic nerve or any of the landmarks to appear. Another invariant might be zooming in, looking through the datset it seemed there were some images that appeared zoomed in where the veins appeared larege. Thsi is another invariant already in the dataset and one we might want our CNN to learn.

[1] Ronneberger, O., Fischer, P., & Brox, T. (2015). U-net: Convolutional networks for biomedical image segmentation. In Lecture Notes in Computer Science (including subseries Lecture Notes in Artificial Intelligence and Lecture Notes in Bioinformatics) (Vol. 9351, pp. 234–241). Springer Verlag. https://doi.org/10.1007/978-3-319-24574-4_28

#### Q2 (11 points)
Check how balanced the dataset is, showing how many negative labels there are for each positive label. Use this information to change the weighting of the positive class in the loss for Question 3 and Question 5, and explain why in some cases using the weighting helps in improving performance.

In [ ]:
# For the given data sets calculates how many values are positive vs negative
# for the mask
def checkDatasetBalance(dataset):
    total_positive = 0
    total_negative = 0
    for batch_idx, (data, segmentation, mask) in enumerate(dataset):
        segmentation_np = segmentation.cpu().numpy()
        mask_np = mask.cpu().numpy()
        positive = np.sum(segmentation_np == 1)
        background = np.sum(mask_np == 0)
        negative = np.sum(segmentation_np == 0) - background
        total_positive += positive
        total_negative += negative 
    return total_positive, total_negative
        

positive, negative = checkDatasetBalance(RetinaDataset(retina_array)) # Without transform
ratio_n_p = negative / positive
print('Total Negative Labels: ' + str(negative))
print('Total Positive Labels: ' + str(positive))
print('Negative labels for each positive: ' + str(ratio_n_p))

### Weighting Explanation
In an unbalanced dataset, the class with more examples contributes more to the loss. By weighting the classes we guarantee that all classes contribute equally to the loss. This improves performance because it forces the network to learn all classes well to improve the loss. In an unbalanced dataset, without weighting applied, the network might learn information about the frequency of the classes as part of its predication reducing the ability of the network to generalize well. 



#### Q3 (45 points)
Build a u-net model class that inherits from the torch.nn.Module class. Follow the model as described in the original paper (https://arxiv.org/pdf/1505.04597.pdf - Fig. 1 and Section 2), but with these modifications chosen to simplify the assignment and to reduce memory use:
- Add 2D batch normalizations between convolutions and ReLUs.
- Add paddings to the convolutions so that the outputs of the convolutions have the same spatial size as the inputs. Because of this, the cropping before the concatenation of the skip connections is not necessary.
- The upsampling convolutions should be coded using the layer torch.nn.ConvTransposed2D. More details to understand what they meant in the paper can be found in the video here, starting at 2:22:  https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/
- Reduce the number of channels of all internal layers dividing them by 8. 
- Input should still have 3 channels, but output should have only one channel (binary output). 
- You can use PyTorch's weight initialization. You do not need to implement the initialization of weights as described in the u-net paper.

Use the masks provided with the dataset to mask your loss. Your loss should only backpropagate through the pixels where the mask has a value of 1, and not backpropagate where the mask is 0. Hyperparameters and methods to use are provided in the code cell below. Use the learning rate scheduler for training too. Your network should be able to get an F1 score of at least 0.75 in the validation set of the provided dataset, using the ``get_score_model`` function. Test your model on the test set.


In [ ]:
# To be reused in U-net
class conv3block(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(conv3block, self).__init__()
        self.conv = torch.nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        self.batch = torch.nn.BatchNorm2d(num_features=out_channels)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.batch(x)
        x = self.relu(x)
        return x
    
class convTran2d(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(convTran2d, self).__init__()
        self.convT = torch.nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, stride=2, kernel_size=4, padding=1)
        
    def forward(self, x):
        x = self.convT(x)
        return x
     
        
class UNet(torch.nn.Module):
    def __init__(self, convBlock, convTranspose, deactivate_skip=False):
        super(UNet, self).__init__()
        self.deactivate_skip = deactivate_skip
        # left side
        self.conv8_1 = convBlock(in_channels=3, out_channels=8)
        self.conv8_2 = convBlock(in_channels=8, out_channels=8)
        self.pool = torch.nn.MaxPool2d(kernel_size=2)
        
        self.conv16_1 = convBlock(in_channels=8, out_channels=16)
        self.conv16_2 = convBlock(in_channels=16, out_channels=16)
        # maxpool
        self.conv32_1 = convBlock(in_channels=16, out_channels=32)
        self.conv32_2 = convBlock(in_channels=32, out_channels=32)
        # maxpool
        self.conv64_1 = convBlock(in_channels=32, out_channels=64)
        self.conv64_2 = convBlock(in_channels=64, out_channels=64)
        # maxpool
        self.conv128_1 = convBlock(in_channels=64, out_channels=128)
        self.conv128_2 = convBlock(in_channels=128, out_channels=128)
        
        # right side
        out_c = 128 if self.deactivate_skip else 64
        self.convT_1 = convTranspose(in_channels=128, out_channels=out_c)
        self.conv64_3 = convBlock(in_channels=128, out_channels=64)
        self.conv64_4 = convBlock(in_channels=64, out_channels=64)
        
        out_c = 64 if self.deactivate_skip else 32 
        self.convT_2 = convTranspose(in_channels=64, out_channels=out_c)
        self.conv32_3 = convBlock(in_channels=64, out_channels=32)
        self.conv32_4 = convBlock(in_channels=32, out_channels=32)
        
        out_c = 32 if self.deactivate_skip else 16 
        self.convT_3 = convTranspose(in_channels=32, out_channels=out_c)
        self.conv16_3 = convBlock(in_channels=32, out_channels=16)
        self.conv16_4 = convBlock(in_channels=16, out_channels=16)
        
        out_c = 16 if self.deactivate_skip else 8 
        self.convT_4 = convTranspose(in_channels=16, out_channels=out_c)
        self.conv8_3 = convBlock(in_channels=16, out_channels=8)
        self.conv8_4 = convBlock(in_channels=8, out_channels=8)
        self.conv1 = convBlock(in_channels=8, out_channels=1)
        

    def forward(self, x):
        # left side
        x = self.conv8_1(x)
        skip_1 = self.conv8_2(x)
        x = self.pool(skip_1)
        
        x = self.conv16_1(x)
        skip_2 = self.conv16_2(x)
        x = self.pool(skip_2)
        
        x = self.conv32_1(x)
        skip_3 = self.conv32_2(x)
        x = self.pool(skip_3)
        
        x = self.conv64_1(x)
        skip_4 = self.conv64_2(x)
        x = self.pool(skip_4)
        
        x = self.conv128_1(x)
        x = self.conv128_2(x)
        
        # right side
        x = self.convT_1(x)
        x = x if self.deactivate_skip else torch.cat((x, skip_4), dim=1) 
        x = self.conv64_3(x)
        x = self.conv64_4(x)
        
        x = self.convT_2(x)
        x = x if self.deactivate_skip else torch.cat((x, skip_3), dim=1) 
        x = self.conv32_3(x)
        x = self.conv32_4(x)
        
        x = self.convT_3(x)
        x = x if self.deactivate_skip else torch.cat((x, skip_2), dim=1) 
        x = self.conv16_3(x)
        x = self.conv16_4(x)
        
        x = self.convT_4(x)
        x = x if self.deactivate_skip else torch.cat((x, skip_1), dim=1) 
        x = self.conv8_3(x)
        x = self.conv8_4(x)
        x = self.conv1(x)
        return x

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

#instantiate your model here:
model = UNet(conv3block, convTran2d)
model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
n_epochs = 200
# Learning rate is reduced after plateauing to stabilize the end of training.
# use the learning rate scheduler as defined here. Example on how to integrate it to training in
# https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.StepLR
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

#train your model here:
# Define loss
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(ratio_n_p))

val_scores = []
train_scores = []
target_score = 0.75
best_score = 0
best_model = copy.deepcopy(model)

print('Starting Training!')
for epoch in range(n_epochs):
    print('\nEpoch: ' + str(epoch))
    scheduler.step()
    
    model.train()
    
    losses = []
    for image, segmentation, mask in train_loader:
        optimizer.zero_grad()
        image = image.cuda()
        segmentation = segmentation.cuda()
        mask = mask.cuda()
        out = model(image)
        loss = criterion(torch.mul(out, mask),  torch.mul(segmentation, mask))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    print('Loss: ' + str(np.mean(losses)))
    
    # Get training data accuracy
    train_score = get_score_model(model, train_loader)
    mean_train_score = np.mean(train_score)
    print('Trainig Score: ' + str(mean_train_score))
    train_scores.append(mean_train_score)
    
    # Get validation data accuracy
    val_score = get_score_model(model, val_loader)
    mean_val_score = np.mean(val_score)
    print('Validation Score: ' + str(mean_val_score))
    val_scores.append(mean_val_score)
    
    # Save Best Model
    if mean_val_score > target_score and mean_val_score > best_score:
        best_score = mean_val_score
        best_model = copy.deepcopy(model)
        print('New Best Model Saved!')
    


In [ ]:
# Plot Training and Validation F1 Score vs Epoch Number
plt.plot(range(n_epochs), train_scores, 'b', range(n_epochs), val_scores, 'r')
plt.title('F1 Scores for UNet Model')
plt.legend(['Training Data', 'Validation Data'])
plt.xlabel('Epoch Number')
plt.ylabel('F1 Score')
plt.show()

### UNet on Test Data Set

In [ ]:
test_score = get_score_model(best_model, test_loader)
print('F1 score for test data: ' + str(test_score))

#### Q4 (8 points)
Visualize a few outputs of your network in the validation set and compare with your ground truth. Comment on the kinds of mistakes that you are able to distinguish.

In [ ]:
# gets example outputs
def get_example_outputs(model, data_loader):
    #toggle model to eval mode
    model.eval()
    ground_truth = []
    model_output = []
    with torch.no_grad():
        logits_predicted = np.zeros([0, 1, 304, 352])
        segmentations = np.zeros([0, 1, 304, 352])

        for image, segmentation, mask  in data_loader:
            image = image.cuda()
            model_output.append(model(image))
            ground_truth.append(segmentation)
            if len(model_output) == 5:
                return model_output, ground_truth;
            

model_out, gt = get_example_outputs(model, val_loader)
for idx in range(5):
    plt.figure()
    plt.title("Model Output")
    model_out[idx] = model_out[idx].view(model_out[idx].shape[2], model_out[idx].shape[3])
    plt.imshow(model_out[idx].cpu().numpy())
#     plt.savefig('img/model'+str(idx)+'.png') # Saved image so I could get better look
    plt.figure()
    plt.title("Segmentation ground truth")
    gt[idx] = gt[idx].view(gt[idx].shape[2], gt[idx].shape[3])
    plt.imshow(gt[idx].cpu().numpy())
#     plt.savefig('img/gt'+str(idx)+'.png') # Saved image so I could get better look

    

### Mistakes I can Identify
In the model output the segmentation lines are much thinner than the ground truth. When the ground truth has disjoint regions or faint lines the output does not have those lines or they are extremely faint or fuzzy and hard to see or identify.

#### Q5 (23 points)
Modify your u-net module to receive a boolean argument in initialization/contruction that can deactivate the use of the skip connections. To compensate for not having the extra channels coming from the skip connection, you should double the number of channels out of the upsampling layer when skip connections are deactivated. Train and test the network without the skip connections, using the hyperparameters as provided in the cell below. You should be able to get an F1 score of at least 0.5.  When compared to the network trained on Question 3, which network performs better? Explain why that happens, detailing the role of the skip connections for a segmentation task. Link your explanations to visualizations of the output of the network without skip connections.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=6, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

#instantiate your model here:
model = UNet(conv3block, convTran2d, deactivate_skip=True)
model = model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
n_epochs = 200
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

#train your model here:
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(ratio_n_p))

val_scores = []
train_scores = []
target_score = 0.50
best_score = 0
best_model = copy.deepcopy(model)

print('Starting Training!')
for epoch in range(n_epochs):
    print('\nEpoch: ' + str(epoch))
    scheduler.step()
    
    model.train()
    
    losses = []
    for image, segmentation, mask in train_loader:
        optimizer.zero_grad()
        image = image.cuda()
        segmentation = segmentation.cuda()
        mask = mask.cuda()
        out = model(image)
        loss = criterion(torch.mul(out, mask),  torch.mul(segmentation, mask))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    print('Loss: ' + str(np.mean(losses)))
    
    # Get training data accuracy
    train_score = get_score_model(model, train_loader)
    mean_train_score = np.mean(train_score)
    print('Trainig Score: ' + str(mean_train_score))
    train_scores.append(mean_train_score)
    
    # Get validation data accuracy
    val_score = get_score_model(model, val_loader)
    mean_val_score = np.mean(val_score)
    print('Validation Score: ' + str(mean_val_score))
    val_scores.append(mean_val_score)
    
    # Save Best Model
    if mean_val_score > target_score and mean_val_score > best_score:
        best_score = mean_val_score
        best_model = copy.deepcopy(model)
        print('New Best Model Saved!')

In [ ]:
# Plot Training and Validation F1 Score vs Epoch Number
plt.plot(range(n_epochs), train_scores, 'b', range(n_epochs), val_scores, 'r')
plt.title('F1 Scores for UNet Model Without Skip Connections')
plt.legend(['Training Data', 'Validation Data'])
plt.xlabel('Epoch Number')
plt.ylabel('F1 Score')
plt.show()

### UNet Without Skip on Test Data Set

In [ ]:
test_score = get_score_model(best_model, test_loader)
print('F1 score for test data: ' + str(test_score))

In [ ]:
# gets example outputs
def get_example_outputs(model, data_loader):
    #toggle model to eval mode
    model.eval()
    ground_truth = []
    model_output = []
    with torch.no_grad():
        for image, segmentation, mask  in data_loader:
            image = image.cuda()
            model_output.append(model(image))
            ground_truth.append(segmentation)
            if len(model_output) == 5:
                return model_output, ground_truth
            

model_out, gt = get_example_outputs(model, val_loader)
for idx in range(5):
    plt.figure()
    plt.title("Model Output")
    model_out[idx] = model_out[idx].view(model_out[idx].shape[2], model_out[idx].shape[3])
    plt.imshow(model_out[idx].cpu().numpy())
#     plt.savefig('img/model_skip'+str(idx)+'.png') # Saved image so I could get better look
    plt.figure()
    plt.title("Segmentation ground truth")
    gt[idx] = gt[idx].view(gt[idx].shape[2], gt[idx].shape[3])
    plt.imshow(gt[idx].cpu().numpy())
#     plt.savefig('img/gt_skip'+str(idx)+'.png') # Saved image so I could get better look

    

### Explanation 
The network in Q3 performed better. 

In the paper [1] the skip connections are explained as improving localization by combining the locality information from the downsampling path with the contextual information in the upsampling path resulting in a more precise output. 

Looking at the model outputs from Q5 we see that the segmentations are considerably blurrier than the segmentations from Q3. Witout the locality information from the skip connection the network seems more uncertain about where the positive lables should be in the image; thus the blurrines.

[1] Ronneberger, O., Fischer, P., & Brox, T. (2015). U-net: Convolutional networks for biomedical image segmentation. In Lecture Notes in Computer Science (including subseries Lecture Notes in Artificial Intelligence and Lecture Notes in Bioinformatics) (Vol. 9351, pp. 234–241). Springer Verlag. https://doi.org/10.1007/978-3-319-24574-4_28